# Import module and data
---

In [1]:
import os
import numpy as np
import pandas as pd
import datetime as datetime

# !pip install wfdb
import wfdb # Waveform Database Software Package (WFDB) for Python
import ast

In [2]:
my_path = os.getcwd()
path = '/home/ubuntu/dr-you-ecg-20220420_mount/STEMI_JKL/PTB_XL'
my_dir = '/home/ubuntu/dr-you-ecg-20220420_mount/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'

# Preprocess
---
### (1) load data

In [4]:
Y = pd.read_csv('/home/ubuntu/dr-you-ecg-20220420_mount/STEMI_JKL/PTB_XL/PTB_label_short.csv') #21837
#Y = Y.drop(['diagnosis'], axis=1)
Y.head()

,ecg_id,patient_id,scp_codes,diagnosis,filename_hr
0,1,15709.0,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",['NORM'],records500/00000/00001_hr
1,2,13243.0,"{'NORM': 80.0, 'SBRAD': 0.0}",['NORM'],records500/00000/00002_hr
2,3,20372.0,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00003_hr
3,4,17014.0,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00004_hr
4,5,17448.0,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00005_hr


In [217]:
df_total = pd.read_csv('/home/ubuntu/dr-you-ecg-20220420_mount/STEMI_JKL/PTB_XL/PTB_label_total.csv')
df_Y = df_total[['patient_id', 'age', 'sex','height', 'weight', 'recording_date', 'scp_codes', 'diagnosis', 'filename_hr', ]]
df_Y.reset_index(drop=True, inplace=True)
df_Y

,patient_id,age,sex,height,weight,recording_date,scp_codes,diagnosis,filename_hr
0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",['NORM'],records500/00000/00001_hr
1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,"{'NORM': 80.0, 'SBRAD': 0.0}",['NORM'],records500/00000/00002_hr
2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00003_hr
3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00004_hr
4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00005_hr
...,...,...,...,...,...,...,...,...,...
21832,17180.0,67.0,1,NaN,NaN,2001-05-31 09:14:35,"{'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...",['NDT'],records500/21000/21833_hr
21833,20703.0,93.0,0,NaN,NaN,2001-06-05 11:33:39,"{'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}",['NORM'],records500/21000/21834_hr
21834,19311.0,59.0,1,NaN,NaN,2001-06-08 10:30:27,"{'ISCAS': 50.0, 'SR': 0.0}",['ISCAS'],records500/21000/21835_hr
21835,8873.0,64.0,1,NaN,NaN,2001-06-09 18:21:49,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/21000/21836_hr


In [15]:
df_total.columns

Index(['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr',
       'diagnostic_superclass', 'diagnosis'],
      dtype='object')

---
### (2) make the label table

In [218]:
# replace '/' to '_' (string '/' causes syntax error) : 21837
# remove data which have uncertain diagnosis or rhythm (only leave 100% likelihood) : 17903
scp_list = []
for i in range(len(df_Y)):
    scp_dict =df_Y.at[i,'scp_codes']
    scp_dict = ast.literal_eval(scp_dict)
    key_list = []
    for key, value in scp_dict.items():
        if value != 0:
            if "/" in key:
                key = key.replace('/','_')
            key_list.append(key)
    scp_list.append(key_list)
    
df_Y['confirm_state'] = scp_list
df_Y

/tmp/ipykernel_10645/2776834220.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Y['confirm_state'] = scp_list


,patient_id,age,sex,height,weight,recording_date,scp_codes,diagnosis,filename_hr,confirm_state
0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",['NORM'],records500/00000/00001_hr,[NORM]
1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,"{'NORM': 80.0, 'SBRAD': 0.0}",['NORM'],records500/00000/00002_hr,[NORM]
2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00003_hr,[NORM]
3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00004_hr,[NORM]
4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00005_hr,[NORM]
...,...,...,...,...,...,...,...,...,...,...
21832,17180.0,67.0,1,NaN,NaN,2001-05-31 09:14:35,"{'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...",['NDT'],records500/21000/21833_hr,"[NDT, PVC]"
21833,20703.0,93.0,0,NaN,NaN,2001-06-05 11:33:39,"{'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}",['NORM'],records500/21000/21834_hr,[NORM]
21834,19311.0,59.0,1,NaN,NaN,2001-06-08 10:30:27,"{'ISCAS': 50.0, 'SR': 0.0}",['ISCAS'],records500/21000/21835_hr,[ISCAS]
21835,8873.0,64.0,1,NaN,NaN,2001-06-09 18:21:49,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/21000/21836_hr,[NORM]


In [219]:
full_index = []
for i in range(len(Y)):
    val_list = Y.iloc[i]['confirm_state']
    if len(val_list) != 0:
        full_index.append(i)

df_Y = df_Y.loc[full_index]
df_Y = df_Y.reset_index(drop=True)
df_Y

,patient_id,age,sex,height,weight,recording_date,scp_codes,diagnosis,filename_hr,confirm_state
0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",['NORM'],records500/00000/00001_hr,[NORM]
1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,"{'NORM': 80.0, 'SBRAD': 0.0}",['NORM'],records500/00000/00002_hr,[NORM]
2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00003_hr,[NORM]
3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00004_hr,[NORM]
4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00005_hr,[NORM]
...,...,...,...,...,...,...,...,...,...,...
21832,17180.0,67.0,1,NaN,NaN,2001-05-31 09:14:35,"{'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...",['NDT'],records500/21000/21833_hr,"[NDT, PVC]"
21833,20703.0,93.0,0,NaN,NaN,2001-06-05 11:33:39,"{'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}",['NORM'],records500/21000/21834_hr,[NORM]
21834,19311.0,59.0,1,NaN,NaN,2001-06-08 10:30:27,"{'ISCAS': 50.0, 'SR': 0.0}",['ISCAS'],records500/21000/21835_hr,[ISCAS]
21835,8873.0,64.0,1,NaN,NaN,2001-06-09 18:21:49,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/21000/21836_hr,[NORM]


In [220]:
df_Y = df_Y.drop(['scp_codes', 'diagnosis'], axis=1)
df_Y

,patient_id,age,sex,height,weight,recording_date,filename_hr,confirm_state
0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,records500/00000/00001_hr,[NORM]
1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,records500/00000/00002_hr,[NORM]
2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,records500/00000/00003_hr,[NORM]
3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,records500/00000/00004_hr,[NORM]
4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,records500/00000/00005_hr,[NORM]
...,...,...,...,...,...,...,...,...
21832,17180.0,67.0,1,NaN,NaN,2001-05-31 09:14:35,records500/21000/21833_hr,"[NDT, PVC]"
21833,20703.0,93.0,0,NaN,NaN,2001-06-05 11:33:39,records500/21000/21834_hr,[NORM]
21834,19311.0,59.0,1,NaN,NaN,2001-06-08 10:30:27,records500/21000/21835_hr,[ISCAS]
21835,8873.0,64.0,1,NaN,NaN,2001-06-09 18:21:49,records500/21000/21836_hr,[NORM]


In [221]:
df_Y

,patient_id,age,sex,height,weight,recording_date,filename_hr,confirm_state
0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,records500/00000/00001_hr,[NORM]
1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,records500/00000/00002_hr,[NORM]
2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,records500/00000/00003_hr,[NORM]
3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,records500/00000/00004_hr,[NORM]
4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,records500/00000/00005_hr,[NORM]
...,...,...,...,...,...,...,...,...
21832,17180.0,67.0,1,NaN,NaN,2001-05-31 09:14:35,records500/21000/21833_hr,"[NDT, PVC]"
21833,20703.0,93.0,0,NaN,NaN,2001-06-05 11:33:39,records500/21000/21834_hr,[NORM]
21834,19311.0,59.0,1,NaN,NaN,2001-06-08 10:30:27,records500/21000/21835_hr,[ISCAS]
21835,8873.0,64.0,1,NaN,NaN,2001-06-09 18:21:49,records500/21000/21836_hr,[NORM]


#### form

In [50]:
agg_df = pd.read_csv('/home/ubuntu/dr-you-ecg-20220420_mount/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/' + 'scp_statements.csv', index_col = 0)
agg_df = agg_df[agg_df.form == 1].reset_index(drop=False)
l_form = agg_df['index'].to_list() #19
len(l_form)

19

In [51]:
scp_list = []
for i in range(len(Y)):
    scp_dict = Y.loc[i,'scp_codes']
    scp_dict = ast.literal_eval(scp_dict)
    key_list = []
    for key, value in scp_dict.items():
        if (value != 0) & (key in l_form):
            key_list.append(key)
    scp_list.append(key_list)

In [52]:
a = []
for i in scp_list:
    if i != []:
        a.append(i)
len(a)

3502

In [53]:
df_a = pd.DataFrame(a)
df_a[0].nunique()

6

#### subgroup

In [222]:
#df_y = df_Y.drop(['rhythm'], axis=1)
df_y = df_Y

In [231]:
agg_df = pd.read_csv('/home/ubuntu/dr-you-ecg-20220420_mount/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/' + 'scp_statements.csv', index_col = 0)
agg_df = agg_df[~agg_df.diagnostic_subclass.isnull()].reset_index(drop=False)
l_form = agg_df['index'].to_list() #44
len(l_form)

44

In [232]:
agg_df = agg_df[['index','diagnostic_subclass']]
#agg_df = agg_df.drop(9, axis=0)

In [234]:
agg_df[agg_df['diagnostic_subclass']=='NST_']

,index,diagnostic_subclass
1,NST_,NST_


In [236]:
agg_df.loc[1, 'diagnostic_subclass'] = 'STTC'
agg_df.reset_index(drop=True, inplace=True)
agg_df

,index,diagnostic_subclass
0,NDT,STTC
1,NST_,STTC
2,DIG,STTC
3,LNGQT,STTC
4,NORM,NORM
5,IMI,IMI
6,ASMI,AMI
7,LVH,LVH
8,LAFB,LAFB/LPFB
9,IRBBB,IRBBB


In [237]:
df_y

,patient_id,age,sex,height,weight,recording_date,filename_hr,confirm_state
0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,records500/00000/00001_hr,[NORM]
1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,records500/00000/00002_hr,[NORM]
2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,records500/00000/00003_hr,[NORM]
3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,records500/00000/00004_hr,[NORM]
4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,records500/00000/00005_hr,[NORM]
...,...,...,...,...,...,...,...,...
21832,17180.0,67.0,1,NaN,NaN,2001-05-31 09:14:35,records500/21000/21833_hr,"[NDT, PVC]"
21833,20703.0,93.0,0,NaN,NaN,2001-06-05 11:33:39,records500/21000/21834_hr,[NORM]
21834,19311.0,59.0,1,NaN,NaN,2001-06-08 10:30:27,records500/21000/21835_hr,[ISCAS]
21835,8873.0,64.0,1,NaN,NaN,2001-06-09 18:21:49,records500/21000/21836_hr,[NORM]


In [238]:
idx_list =[]
for idx in agg_df['index']:
    if "/" in idx:
        idx = idx.replace('/','_')
    idx_list.append(idx)
agg_df['re'] = idx_list
agg_df.index = idx_list
agg_df

,index,diagnostic_subclass,re
NDT,NDT,STTC,NDT
NST_,NST_,STTC,NST_
DIG,DIG,STTC,DIG
LNGQT,LNGQT,STTC,LNGQT
NORM,NORM,NORM,NORM
IMI,IMI,IMI,IMI
ASMI,ASMI,AMI,ASMI
LVH,LVH,LVH,LVH
LAFB,LAFB,LAFB/LPFB,LAFB
IRBBB,IRBBB,IRBBB,IRBBB


In [239]:
a = agg_df[agg_df['index']=='NORM'].diagnostic_subclass
print(a)

NORM    NORM
Name: diagnostic_subclass, dtype: object


In [102]:
tmp = []
for state in df_y.confirm_state:
    for each in state:
        if each in agg_df.index:
            tmp.append(agg_df.loc[each].diagnostic_subclass)
tmp_ls = list(set(tmp))

In [111]:
def aggregate_state(y_ls):
    tmp = []
    for state in y_ls:
        for each in state:
            if each in agg_df.index:
                tmp.append(agg_df.loc[each].diagnostic_subclass)
    return list(set(tmp))

In [241]:
def aggregate_state(y_ls):
    tmp = []
    for state in y_ls:
        if state in agg_df.index:
            tmp.append(agg_df.loc[state].diagnostic_subclass)
    return list(set(tmp))

In [242]:
df_y['diagnostic_subclass'] = df_y.confirm_state.apply(aggregate_state)
df_y

,patient_id,age,sex,height,weight,recording_date,filename_hr,confirm_state,diagnostic_subclass
0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,records500/00000/00001_hr,[NORM],[NORM]
1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,records500/00000/00002_hr,[NORM],[NORM]
2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,records500/00000/00003_hr,[NORM],[NORM]
3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,records500/00000/00004_hr,[NORM],[NORM]
4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,records500/00000/00005_hr,[NORM],[NORM]
...,...,...,...,...,...,...,...,...,...
21832,17180.0,67.0,1,NaN,NaN,2001-05-31 09:14:35,records500/21000/21833_hr,"[NDT, PVC]",[STTC]
21833,20703.0,93.0,0,NaN,NaN,2001-06-05 11:33:39,records500/21000/21834_hr,[NORM],[NORM]
21834,19311.0,59.0,1,NaN,NaN,2001-06-08 10:30:27,records500/21000/21835_hr,[ISCAS],[ISCA]
21835,8873.0,64.0,1,NaN,NaN,2001-06-09 18:21:49,records500/21000/21836_hr,[NORM],[NORM]


In [243]:
full_index = []
for i in range(len(df_y)):
    val_list =df_y.iloc[i]['diagnostic_subclass']
    if len(val_list) != 0:
        full_index.append(i)

len(full_index)

21380

In [244]:
df_full = df_y.loc[full_index]
df_full.reset_index(drop=False, inplace=True)
df_full = df_full.rename(columns={'index':'ecg_id'})
df_full

,ecg_id,patient_id,age,sex,height,weight,recording_date,filename_hr,confirm_state,diagnostic_subclass
0,0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,records500/00000/00001_hr,[NORM],[NORM]
1,1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,records500/00000/00002_hr,[NORM],[NORM]
2,2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,records500/00000/00003_hr,[NORM],[NORM]
3,3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,records500/00000/00004_hr,[NORM],[NORM]
4,4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,records500/00000/00005_hr,[NORM],[NORM]
...,...,...,...,...,...,...,...,...,...,...
21375,21832,17180.0,67.0,1,NaN,NaN,2001-05-31 09:14:35,records500/21000/21833_hr,"[NDT, PVC]",[STTC]
21376,21833,20703.0,93.0,0,NaN,NaN,2001-06-05 11:33:39,records500/21000/21834_hr,[NORM],[NORM]
21377,21834,19311.0,59.0,1,NaN,NaN,2001-06-08 10:30:27,records500/21000/21835_hr,[ISCAS],[ISCA]
21378,21835,8873.0,64.0,1,NaN,NaN,2001-06-09 18:21:49,records500/21000/21836_hr,[NORM],[NORM]


In [245]:
path

'/home/ubuntu/dr-you-ecg-20220420_mount/STEMI_JKL/PTB_XL'

In [246]:
df_full['patient_id'].nunique()

18599

In [247]:
df_full.head()

,ecg_id,patient_id,age,sex,height,weight,recording_date,filename_hr,confirm_state,diagnostic_subclass
0,0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,records500/00000/00001_hr,[NORM],[NORM]
1,1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,records500/00000/00002_hr,[NORM],[NORM]
2,2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,records500/00000/00003_hr,[NORM],[NORM]
3,3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,records500/00000/00004_hr,[NORM],[NORM]
4,4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,records500/00000/00005_hr,[NORM],[NORM]


In [248]:
#df_full.to_csv(path+'/ptb-xl_subgroup_v2.csv', index=False)

In [214]:
#df_full = pd.read_csv(path+'/ptb-xl_subgroup.csv')

In [249]:
df_full

,ecg_id,patient_id,age,sex,height,weight,recording_date,filename_hr,confirm_state,diagnostic_subclass
0,0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,records500/00000/00001_hr,[NORM],[NORM]
1,1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,records500/00000/00002_hr,[NORM],[NORM]
2,2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,records500/00000/00003_hr,[NORM],[NORM]
3,3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,records500/00000/00004_hr,[NORM],[NORM]
4,4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,records500/00000/00005_hr,[NORM],[NORM]
...,...,...,...,...,...,...,...,...,...,...
21375,21832,17180.0,67.0,1,NaN,NaN,2001-05-31 09:14:35,records500/21000/21833_hr,"[NDT, PVC]",[STTC]
21376,21833,20703.0,93.0,0,NaN,NaN,2001-06-05 11:33:39,records500/21000/21834_hr,[NORM],[NORM]
21377,21834,19311.0,59.0,1,NaN,NaN,2001-06-08 10:30:27,records500/21000/21835_hr,[ISCAS],[ISCA]
21378,21835,8873.0,64.0,1,NaN,NaN,2001-06-09 18:21:49,records500/21000/21836_hr,[NORM],[NORM]


In [250]:
a = []
for state in df_full.diagnostic_subclass:
    for i in state:
        if i not in a:
            a.append(i)
a

['NORM',
 'IMI',
 'STTC',
 'LVH',
 'LAFB/LPFB',
 'RAO/RAE',
 'IRBBB',
 'RVH',
 'IVCD',
 'LMI',
 'AMI',
 'ISCA',
 '_AVB',
 'SEHYP',
 'ISCI',
 'CRBBB',
 'CLBBB',
 'LAO/LAE',
 'ILBBB',
 'WPW',
 'PMI']

In [252]:
print(a)

['NORM', 'IMI', 'STTC', 'LVH', 'LAFB/LPFB', 'RAO/RAE', 'IRBBB', 'RVH', 'IVCD', 'LMI', 'AMI', 'ISCA', '_AVB', 'SEHYP', 'ISCI', 'CRBBB', 'CLBBB', 'LAO/LAE', 'ILBBB', 'WPW', 'PMI']


In [251]:
len(a)

21

#### rhythm (12)

In [22]:
agg_df = pd.read_csv('/home/ubuntu/dr-you-ecg-20220420_mount/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/' + 'scp_statements.csv', index_col = 0)
agg_df = agg_df[agg_df.rhythm == 1].reset_index(drop=False)
l_rhythm = agg_df['index'].to_list() #12

In [41]:
scp_list = []
for i in range(len(Y)):
    scp_dict = Y.loc[i,'scp_codes']
    scp_dict = ast.literal_eval(scp_dict)
    key_list = []
    for key, value in scp_dict.items():
        if (value != 0) & (key in l_rhythm):
            key_list.append(key)
    scp_list.append(key_list)

In [46]:
a = []
for i in scp_list:
    if i != []:
        a.append(i)
len(a)

445

In [49]:
#df_a = pd.DataFrame(a)
df_a[0].nunique()

6

In [43]:
Y['rhythm'] = scp_list

full_index = []
for i in range(len(Y)):
    val_list = Y.iloc[i]['confirm_state']
    if len(val_list) != 0:
        full_index.append(i)

Y = Y.loc[full_index]
Y = Y.reset_index(drop=True)
Y

,ecg_id,patient_id,scp_codes,diagnosis,filename_hr,confirm_state,rhythm
0,1,15709.0,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",['NORM'],records500/00000/00001_hr,[NORM],[]
1,2,13243.0,"{'NORM': 80.0, 'SBRAD': 0.0}",['NORM'],records500/00000/00002_hr,[NORM],[]
2,3,20372.0,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00003_hr,[NORM],[]
3,4,17014.0,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00004_hr,[NORM],[]
4,5,17448.0,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/00000/00005_hr,[NORM],[]
...,...,...,...,...,...,...,...
21832,21833,17180.0,"{'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...",['NDT'],records500/21000/21833_hr,"[NDT, PVC]",[]
21833,21834,20703.0,"{'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}",['NORM'],records500/21000/21834_hr,[NORM],[]
21834,21835,19311.0,"{'ISCAS': 50.0, 'SR': 0.0}",['ISCAS'],records500/21000/21835_hr,[ISCAS],[]
21835,21836,8873.0,"{'NORM': 100.0, 'SR': 0.0}",['NORM'],records500/21000/21836_hr,[NORM],[]


---
### (3) make the y_dataset (by sklearn.MultiLabelBinarizer)

In [253]:
df_full

,ecg_id,patient_id,age,sex,height,weight,recording_date,filename_hr,confirm_state,diagnostic_subclass
0,0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,records500/00000/00001_hr,[NORM],[NORM]
1,1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,records500/00000/00002_hr,[NORM],[NORM]
2,2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,records500/00000/00003_hr,[NORM],[NORM]
3,3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,records500/00000/00004_hr,[NORM],[NORM]
4,4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,records500/00000/00005_hr,[NORM],[NORM]
...,...,...,...,...,...,...,...,...,...,...
21375,21832,17180.0,67.0,1,NaN,NaN,2001-05-31 09:14:35,records500/21000/21833_hr,"[NDT, PVC]",[STTC]
21376,21833,20703.0,93.0,0,NaN,NaN,2001-06-05 11:33:39,records500/21000/21834_hr,[NORM],[NORM]
21377,21834,19311.0,59.0,1,NaN,NaN,2001-06-08 10:30:27,records500/21000/21835_hr,[ISCAS],[ISCA]
21378,21835,8873.0,64.0,1,NaN,NaN,2001-06-09 18:21:49,records500/21000/21836_hr,[NORM],[NORM]


In [254]:
y_list = df_full['diagnostic_subclass'].values.tolist()
y_list[:5]

[['NORM'], ['NORM'], ['NORM'], ['NORM'], ['NORM']]

In [255]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(df_full['diagnostic_subclass'].values)
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [256]:
labels[39,:]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0])

In [257]:
# labels
labels.shape

(21380, 21)

In [258]:
# classes (50)
mlb.classes_ 

array(['AMI', 'CLBBB', 'CRBBB', 'ILBBB', 'IMI', 'IRBBB', 'ISCA', 'ISCI',
       'IVCD', 'LAFB/LPFB', 'LAO/LAE', 'LMI', 'LVH', 'NORM', 'PMI',
       'RAO/RAE', 'RVH', 'SEHYP', 'STTC', 'WPW', '_AVB'], dtype=object)

In [259]:
len(mlb.classes_ )

21

In [261]:
# save the y_dataset 
np.save('/home/ubuntu/dr-you-ecg-20220420_mount/STEMI_JKL/PTB_XL/ptbxl_Y.npy',labels)

In [262]:
df_full.columns

Index(['ecg_id', 'patient_id', 'age', 'sex', 'height', 'weight',
       'recording_date', 'filename_hr', 'confirm_state',
       'diagnostic_subclass'],
      dtype='object')

In [263]:
col_list = df_full.columns[::-1]
col_list

Index(['diagnostic_subclass', 'confirm_state', 'filename_hr', 'recording_date',
       'weight', 'height', 'sex', 'age', 'patient_id', 'ecg_id'],
      dtype='object')

In [264]:
df_ptb = pd.DataFrame(columns=mlb.classes_, data=labels)
df_ptb.sum()

AMI          3086
CLBBB         536
CRBBB         542
ILBBB          77
IMI          3281
IRBBB        1118
ISCA          944
ISCI          398
IVCD          789
LAFB/LPFB    1800
LAO/LAE       427
LMI           201
LVH          2137
NORM         9528
PMI            17
RAO/RAE        99
RVH           126
SEHYP          30
STTC         2780
WPW            80
_AVB          827
dtype: int64

In [266]:
# make a label table
df_ptb = pd.DataFrame(columns=mlb.classes_, data=labels)
for i in col_list:
    df_ptb.insert(0, i, df_full[i])

df_ptb.head()

,ecg_id,patient_id,age,sex,height,weight,recording_date,filename_hr,confirm_state,diagnostic_subclass,...,LMI,LVH,NORM,PMI,RAO/RAE,RVH,SEHYP,STTC,WPW,_AVB
0,0,15709.0,56.0,1,NaN,63.0,1984-11-09 09:17:34,records500/00000/00001_hr,[NORM],[NORM],...,0,0,1,0,0,0,0,0,0,0
1,1,13243.0,19.0,0,NaN,70.0,1984-11-14 12:55:37,records500/00000/00002_hr,[NORM],[NORM],...,0,0,1,0,0,0,0,0,0,0
2,2,20372.0,37.0,1,NaN,69.0,1984-11-15 12:49:10,records500/00000/00003_hr,[NORM],[NORM],...,0,0,1,0,0,0,0,0,0,0
3,3,17014.0,24.0,0,NaN,82.0,1984-11-15 13:44:57,records500/00000/00004_hr,[NORM],[NORM],...,0,0,1,0,0,0,0,0,0,0
4,4,17448.0,19.0,1,NaN,70.0,1984-11-17 10:43:15,records500/00000/00005_hr,[NORM],[NORM],...,0,0,1,0,0,0,0,0,0,0


In [267]:
# save the label table
df_ptb.to_csv('/home/ubuntu/dr-you-ecg-20220420_mount/STEMI_JKL/PTB_XL/ptbxl_MasterTable.csv', index=False)

In [270]:
# check
df_ptb.loc[173]

ecg_id                                       180
patient_id                               21551.0
age                                         73.0
sex                                            0
height                                       NaN
weight                                       NaN
recording_date               1986-03-16 08:17:31
filename_hr            records500/00000/00181_hr
confirm_state           [ASMI, ALMI, IMI, ANEUR]
diagnostic_subclass             [STTC, AMI, IMI]
AMI                                            1
CLBBB                                          0
CRBBB                                          0
ILBBB                                          0
IMI                                            1
IRBBB                                          0
ISCA                                           0
ISCI                                           0
IVCD                                           0
LAFB/LPFB                                      0
LAO/LAE             

---
### (4) make the x_dataset

# 여기임 (1/25/2023)

In [283]:
def load_raw_data(df, sampling_rate, path):
    data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = my_dir
sampling_rate=500

# Load raw signal data
X = load_raw_data(df_ptb, sampling_rate, path)

In [284]:
X

array([[[-0.115, -0.05 ,  0.065, ..., -0.035, -0.035, -0.075],
        [-0.115, -0.05 ,  0.065, ..., -0.035, -0.035, -0.075],
        [-0.115, -0.05 ,  0.065, ..., -0.035, -0.035, -0.075],
        ...,
        [ 0.21 ,  0.205, -0.005, ...,  0.185,  0.17 ,  0.18 ],
        [ 0.21 ,  0.205, -0.005, ...,  0.185,  0.17 ,  0.18 ],
        [ 0.21 ,  0.205, -0.005, ...,  0.185,  0.17 ,  0.18 ]],

       [[-0.015,  0.12 ,  0.135, ...,  0.16 ,  0.06 ,  0.065],
        [-0.015,  0.12 ,  0.135, ...,  0.16 ,  0.06 ,  0.065],
        [-0.015,  0.12 ,  0.135, ...,  0.16 ,  0.06 ,  0.065],
        ...,
        [-0.055,  0.015,  0.07 , ..., -0.215, -0.195, -0.13 ],
        [-0.055,  0.015,  0.07 , ..., -0.215, -0.195, -0.13 ],
        [-0.055,  0.015,  0.07 , ..., -0.215, -0.195, -0.13 ]],

       [[-0.035, -0.07 , -0.035, ..., -0.1  , -0.075, -0.065],
        [-0.035, -0.07 , -0.035, ..., -0.1  , -0.075, -0.065],
        [-0.035, -0.07 , -0.035, ..., -0.1  , -0.075, -0.065],
        ...,
        [-0.

In [285]:
len(X)

21380

In [286]:
# 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11
# I, II, III, aVL, aVR, aVF, V1-V6
X_dataset = []
for i in X:
    df_ecg = pd.DataFrame(i)
    df_lead = df_ecg[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
    df_numpy = df_lead.to_numpy()
    X_dataset.append(df_numpy)

In [287]:
X_dataset = np.array(X_dataset)
X_dataset.shape

(21380, 5000, 12)

In [288]:
# x_dataset (2023.01.25)
np.save('/home/ubuntu/dr-you-ecg-20220420_mount/STEMI_JKL/PTB_XL/ptbxl_X.npy',X_dataset)